In [248]:
import pandas as pd
import yfinance as yf

#dfSPY=yf.download("^GSPC",start='2011-01-05', end='2021-01-05')
dfSPY=yf.download("^RUI",start='2011-01-05', end='2021-01-05')
#dfSPY=yf.download("AMZN",start='2011-01-05', end='2021-01-05')

dfSPY=dfSPY[dfSPY.High!=dfSPY.Low]
dfSPY.reset_index(inplace=True)
dfSPY.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-01-05,704.739990,707.419983,699.049988,706.869995,706.869995,0
1,2011-01-06,702.380005,707.929993,700.679993,705.479980,705.479980,0
2,2011-01-07,706.830017,707.929993,698.780029,704.320007,704.320007,0
3,2011-01-10,703.229980,704.770020,699.020020,703.760010,703.760010,0
4,2011-01-11,705.090027,707.929993,703.859985,706.520020,706.520020,0


In [249]:
import pandas_ta as ta
dfSPY['EMA']=ta.ema(dfSPY.Close, length=200)#sma ema
dfSPY['EMA2']=ta.ema(dfSPY.Close, length=150)#sma ema
dfSPY['RSI']=ta.rsi(dfSPY.Close, length=12)
#dfSPY.ta.indicators()
#help(ta.bbands)
my_bbands = ta.bbands(dfSPY.Close, length=14, std=2.0)
my_bbands[0:50]
dfSPY=dfSPY.join(my_bbands)
dfSPY.dropna(inplace=True)
dfSPY.reset_index(inplace=True)
dfSPY[420:425]

,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA,EMA2,RSI,BBL_14_2.0,BBM_14_2.0,BBU_14_2.0,BBB_14_2.0,BBP_14_2.0
420,619,2013-06-24,879.289978,881.609985,863.799988,871.000000,871.000000,0,843.446264,857.944547,34.584329,872.796667,897.986428,923.176189,5.610277,-0.035663
421,620,2013-06-25,870.909973,882.559998,870.909973,879.539978,879.539978,0,843.805406,858.230579,40.230672,870.393533,897.150713,923.907893,5.964924,0.170916
422,621,2013-06-26,879.820007,890.020020,879.820007,887.969971,887.969971,0,844.244854,858.624479,45.313705,869.217057,896.352853,923.488649,6.054713,0.345538
423,622,2013-06-27,890.409973,897.500000,890.409973,894.150024,894.150024,0,844.741423,859.095016,48.796274,869.205250,895.182853,921.160456,5.803865,0.480121
424,623,2013-06-28,892.989990,895.809998,887.580017,890.669983,890.669983,0,845.198424,859.513227,46.959216,869.120133,893.788565,918.456998,5.519970,0.436790


In [250]:
"""def addemasignal(df, backcandles):
    emasignal = [0]*len(df)
    for row in range(backcandles, len(df)):
        upt = 1
        dnt = 1
        for i in range(row-backcandles, row+1):
            if df.High[i]>=df.EMA[i]:
                dnt=0
            if df.Low[i]<=df.EMA[i]:
                upt=0
        if upt==1 and dnt==1:
            #print("!!!!! check trend loop !!!!")
            emasignal[row]=3
        elif upt==1:
            emasignal[row]=2
        elif dnt==1:
            emasignal[row]=1
    df['EMASignal'] = emasignal"""

def addemasignal(df):
    emasignal = [0]*len(df)
    for i in range(0, len(df)):
        if df.EMA2[i]>df.EMA[i]:
            emasignal[i]=2
        elif df.EMA2[i]<df.EMA[i]:
            emasignal[i]=1
    df['EMASignal'] = emasignal
addemasignal(dfSPY)

In [251]:
def addorderslimit(df, percent):
    ordersignal=[0]*len(df)
    for i in range(1, len(df)): #EMASignal of previous candle!!! modified!!!
        if  df.Close[i]<=df['BBL_14_2.0'][i] and df.EMASignal[i]==2:
            ordersignal[i]=df.Close[i]-df.Close[i]*percent
        elif df.Close[i]>=df['BBU_14_2.0'][i] and df.EMASignal[i]==1:
            ordersignal[i]=df.Close[i]+df.Close[i]*percent
    df['ordersignal']=ordersignal
    
addorderslimit(dfSPY, 0.000)

In [252]:
dfSPY[dfSPY.ordersignal!=0]

,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA,EMA2,RSI,BBL_14_2.0,BBM_14_2.0,BBU_14_2.0,BBB_14_2.0,BBP_14_2.0,EMASignal,ordersignal
6,205,2011-10-27,689.780029,716.859985,689.780029,709.609985,709.609985,0,706.228622,689.079461,66.823012,646.911677,674.737858,702.564040,8.247998,1.126606,1,709.609985
73,272,2012-02-03,737.099976,745.580017,737.099976,745.390015,745.390015,0,702.673092,695.584960,77.311981,715.305806,728.690709,742.075611,3.673685,1.123811,1,745.390015
74,273,2012-02-06,743.900024,745.390015,741.299988,745.119995,745.119995,0,703.095449,696.241053,76.847768,717.548107,730.889993,744.231879,3.650860,1.033283,1,745.119995
99,298,2012-03-13,761.450012,773.030029,761.450012,772.979980,772.979980,0,714.405224,712.705857,69.907702,744.743942,757.327144,769.910346,3.323056,1.121973,1,772.979980
117,316,2012-04-09,769.919983,769.919983,761.909973,764.270020,764.270020,0,724.617009,726.293510,42.521546,766.089040,776.452859,786.816678,2.669530,-0.087758,2,764.270020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2143,2342,2020-04-29,1579.069946,1626.739990,1579.069946,1618.599976,1618.599976,0,1627.254578,1623.088955,62.446213,1489.112272,1549.214286,1609.316299,7.759032,1.077233,1,1618.599976
2169,2368,2020-06-05,1725.540039,1778.359985,1725.540039,1767.939941,1767.939941,0,1629.117549,1627.087559,75.883795,1586.809881,1674.752842,1762.695804,10.502202,1.029816,1,1767.939941
2170,2369,2020-06-08,1767.510010,1791.510010,1767.510010,1791.380005,1791.380005,0,1630.732100,1629.263618,78.160733,1584.085380,1686.389988,1788.694597,12.132972,1.013125,1,1791.380005
2245,2444,2020-09-23,1846.260010,1849.959961,1799.790039,1801.829956,1801.829956,0,1735.271364,1758.079711,35.708018,1804.880625,1863.932146,1922.983668,6.336231,-0.025831,2,1801.829956


# Visualization

In [253]:
import numpy as np
def pointposbreak(x):
    if x['ordersignal']!=0:
        return x['ordersignal']
    else:
        return np.nan
dfSPY['pointposbreak'] = dfSPY.apply(lambda row: pointposbreak(row), axis=1)

In [254]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = dfSPY[1000:3200].copy()
#dfpl=dfpl.drop(columns=['level_0'])#!!!!!!!!!!
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=2), name="EMA"),
                go.Scatter(x=dfpl.index, y=dfpl.EMA2, line=dict(color='yellow', width=2), name="EMA2"),        
                go.Scatter(x=dfpl.index, y=dfpl['BBL_14_2.0'], line=dict(color='blue', width=1), name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_14_2.0'], line=dict(color='blue', width=1), name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=6, color="MediumPurple"),
                name="Signal")
#fig.update(layout_yaxis_range = [300,420])
fig.update_xaxes(rangeslider_visible=False)
fig.update_layout(autosize=False, width=600, height=600,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="white")
fig.show()

In [255]:
dfpl = dfSPY[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [266]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.99
    mysize = initsize
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 2
        perc = 0.02
        
        if len(self.trades)>0:
            if self.data.index[-1]-self.trades[-1].entry_time>=10:
                self.trades[-1].close()
            if self.trades[-1].is_long and self.data.RSI[-1]>=75:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=25:
                self.trades[-1].close()
        
        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:  
            sl1 = min(self.data.Low[-1],self.data.Low[-2])*(1-perc)
            tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:         
            sl1 = sl1 = max(self.data.High[-1],self.data.High[-2])*(1+perc)
            tp1 = self.data.Close[-1]-(sl1 - self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=1000, margin=1/5, commission=.000)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning:

Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    2315.0
Duration                               2315.0
Exposure Time [%]                   26.079447
Equity Final [$]                 28728.464895
Equity Peak [$]                  28728.464895
Return [%]                         2772.84649
Buy & Hold Return [%]              213.320124
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -48.245971
Avg. Drawdown [%]                   -7.787384
Max. Drawdown Duration                  334.0
Avg. Drawdown Duration              32.407407
# Trades                                 82.0
Win Rate [%]                        59.756098
Best Trade [%]                       5.857944
Worst Trade [%]                     -5.270279
Avg. Trade [%]                    

In [267]:
bt.plot()

Row(id='39141', ...)